In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation, Input, Dropout, BatchNormalization
from keras import optimizers
import numpy as np
from agnews_utils import get_data, tokenizing, get_matrix

# matrix_path = 's3://obs-wzhtest/cifar/glove.6B/glove.6B.100d.txt'
matrix_path = 's3://obs-wzhtest/cifar/glove.6B/glove.6B.100d.txt'
train_data_path = 'ag-news/ag_news_csv/train.csv'
test_data_path = 'ag-news/ag_news_csv/test.csv'

max_num_words = 20000
max_sequence_length = 150
embedding_dim = 100
batch_size = 500

x_train, y_train, x_test, y_test = get_data(train_data_path, test_data_path)
word_index, x_train, y_train, x_test, y_test = tokenizing(x_train, y_train, x_test, y_test, max_num_words, max_sequence_length)
embedding_matrix = get_matrix(matrix_path, word_index, max_num_words, embedding_dim)

Loading data complete.
shape of x_train tensor:  (120000, 150)
shape of y_train tensor:  (120000, 4)
shape of x_test tensor:  (7600, 150)
shape of y_test tensor:  (7600, 4)
Tokenizing complete.
embedding_matrix shape:  (20001, 100)


In [4]:
# LSTM

num_words = 20000
embedding_layer = Embedding(num_words + 1, embedding_dim, weights=[embedding_matrix],
                           input_length=max_sequence_length)
adam = optimizers.Adam(lr=0.05)


print('Build model...')
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='softmax'))
model.layers[1].trainable = False

model.compile(loss='categorical_crossentropy', optimizer=adam,
             metrics=['accuracy'])
print('Training...')
model.fit(x_train, y_train, batch_size=batch_size, epochs=5,
         validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

print('Test score: ', score)
print('Test accuracy: ', acc)

Build model...
Training...
Train on 120000 samples, validate on 7600 samples
Epoch 1/5
120000/120000 [==============================] - 59s 491us/step - loss: 0.5701 - acc: 0.7990 - val_loss: 0.3778 - val_acc: 0.8861
Epoch 2/5
120000/120000 [==============================] - 56s 468us/step - loss: 0.2977 - acc: 0.9068 - val_loss: 0.3633 - val_acc: 0.8967
Epoch 3/5
120000/120000 [==============================] - 56s 465us/step - loss: 0.2231 - acc: 0.9287 - val_loss: 0.3582 - val_acc: 0.8941
Epoch 4/5
120000/120000 [==============================] - 56s 467us/step - loss: 0.1826 - acc: 0.9408 - val_loss: 0.3835 - val_acc: 0.8967
Epoch 5/5
7600/7600 [==============================] - 1s 172us/step
Test score:  0.3919618063067135
Test accuracy:  0.8951315809237329


In [3]:
max_num_words = 20000
max_sequence_length = 100
embedding_dim = 100
batch_size = 500

x_train, y_train, x_test, y_test = get_data(train_data_path, test_data_path)
word_index, x_train, y_train, x_test, y_test = tokenizing(x_train, y_train, x_test, y_test, max_num_words, max_sequence_length)
embedding_matrix = get_matrix(matrix_path, word_index, max_num_words, embedding_dim)

# LSTM

num_words = 20000
embedding_layer = Embedding(num_words + 1, embedding_dim, weights=[embedding_matrix],
                           input_length=max_sequence_length, dropout=0.2)
print('Build model...')

# sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
adam = optimizers.Adam(lr=0.05)
sgd = optimizers.SGD(lr=0.05, decay=1e-3)

model = Sequential()
model.add(embedding_layer)
# model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(4, activation='softmax'))
model.layers[1].trainable = False

model.compile(loss='categorical_crossentropy', optimizer=adam,
             metrics=['accuracy'])
print('Training...')
model.fit(x_train, y_train, batch_size=batch_size, epochs=10,
         validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

print('Test score: ', score)
print('Test accuracy: ', acc)

x_train length:  120000
y_train length:  120000
x_test length:  7600
y_test length:  7600
Loading data complete.
Tokenizing...
x_all shape:  127600
Found 72002 unique tokens.
shape of x_train tensor:  (120000, 100)
shape of y_train tensor:  (120000, 4)
shape of x_test tensor:  (7600, 100)
shape of y_test tensor:  (7600, 4)
Found 400000 word vectors.
10000 word_index complete.
20000 word_index complete.
embedding_matrix shape:  (20001, 100)


/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.


Build model...


/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, dropout=0.2, recurrent_dropout=0.2)`


Training...
Train on 120000 samples, validate on 7600 samples
Epoch 1/10
120000/120000 [==============================] - 38s 320us/step - loss: 0.5211 - acc: 0.8108 - val_loss: 0.3320 - val_acc: 0.8905
Epoch 2/10
120000/120000 [==============================] - 37s 311us/step - loss: 0.2632 - acc: 0.9131 - val_loss: 0.3384 - val_acc: 0.8925
Epoch 3/10
120000/120000 [==============================] - 38s 313us/step - loss: 0.2002 - acc: 0.9331 - val_loss: 0.3684 - val_acc: 0.8883
Epoch 4/10
120000/120000 [==============================] - 37s 312us/step - loss: 0.1658 - acc: 0.9439 - val_loss: 0.3895 - val_acc: 0.8882
Epoch 5/10
120000/120000 [==============================] - 38s 313us/step - loss: 0.1407 - acc: 0.9531 - val_loss: 0.4169 - val_acc: 0.8876
Epoch 6/10
120000/120000 [==============================] - 38s 314us/step - loss: 0.1274 - acc: 0.9566 - val_loss: 0.4219 - val_acc: 0.8858
Epoch 7/10
120000/120000 [==============================] - 38s 313us/step - loss: 0.1150 - 

In [ ]:
# TextCNN

num_words = 20000
embedding_layer = Embedding(num_words + 1, embedding_dim, weights=[embedding_matrix],
                           input_length=max_sequence_length)
adam = optimizers.Adam(lr=0.001)


print('Build model...')
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='softmax'))
model.layers[1].trainable = False

model.compile(loss='categorical_crossentropy', optimizer=adam,
             metrics=['accuracy'])
print('Training...')
model.fit(x_train, y_train, batch_size=batch_size, epochs=5,
         validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

print('Test score: ', score)
print('Test accuracy: ', acc)